[View in Colaboratory](https://colab.research.google.com/github/TattooeDeer/API_REST_TingoID/blob/master/P2.ipynb)

# Pregunta 2


In [4]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import numpy as np
np.random.seed(11235813)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
from scipy.stats import bernoulli

# Tensorflow & Keras imports
import tensorflow as tf
from keras.layers import Input, RepeatVector, TimeDistributed, Dense, Embedding, Flatten, Activation, Permute, Lambda, CuDNNGRU
from keras.models import Model
from keras import backend as K
from keras.preprocessing import sequence
from keras.models import load_model


# model saving
!sudo apt-get install libhdf5-serial-dev
import h5py

from google.colab import files
!git clone https://github.com/TattooeDeer/T3-ANN.git
%cd T3-ANN
!ls


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/bin/sh: 1: sudo: not found
Cloning into 'T3-ANN'...
remote: Counting objects: 32, done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (32/32), done.
/content/T3-ANN/T3-ANN
att.h5	 Enunciado_T3.ipynb  P2.ipynb	Tarea3.ipynb  train_Q-A.csv
attW.h5  LICENSE	     README.md	test_Q.csv


## a) Carga de los datos en el entorno y análisis descriptivo

In [5]:
train = pd.read_csv('train_Q-A.csv')
test = pd.read_csv('test_Q.csv')

print('Train shape: {0}'.format(train.shape))
print('Test shape: {0}'.format(test.shape))

Train shape: (86821, 3)
Test shape: (11873, 2)


In [6]:
train.head()

,id,question,answer
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,in the late 1990s
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,singing and dancing
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,2003
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,late 1990s


In [7]:
train.describe()

,id,question,answer
count,86821,86821,86821
unique,86821,86769,64763
top,570e469c0dc6ce1900204f03,How old was William IV when he died?,three
freq,1,2,231


Al parecer el primer problema que observamos es que para preguntas cuya respuesta es cuantitativa, la respuesta dada es expresada en algunos casos en palabras y en otros en números, nuestro primer intento de solucionar esto será explorar si se cumplen patrones para cada tipo de respuesta, por ejemplo, solo las fechas se responden con números u otra regla similar.


In [8]:
test.describe()

,id,question
count,11873,11873
unique,11873,11864
top,5729f5a03f37b31900478606,Who conceptualized the piston?
freq,1,2


Se observa algo interesante analizando la cantidad de valores únicos y de conteo de las columnas `question` y  `answer`: La cantidad de ocurrencias únicas no es igual a la cantidad de registros totales, lo que puede indicar que quizás hay preguntas/respuestas repetidas en el dataset. Se observa un comportamiento similar en el conjunto de test aunque en mucho menor medida

In [9]:
print('10 preguntas más populares:\n')
print(train["question"].value_counts().head(10))
print('\n ----------------------------------------------------------\n')
print('10 respuestas más populares:\n')
print(train["answer"].value_counts().head(10))
print('\n ----------------------------------------------------------\n')
print('10 preguntas más populares (Conjunto de test):\n')
print(test["question"].value_counts().head(10))






10 preguntas más populares:

How old was William IV when he died?              2
On what date was the city liberated?              2
What year did Gladstone retire?                   2
What type of religion is Buddhism?                2
Who did Victoria marry?                           2
What is the national bird of Bermuda?             2
What was the name of Alonso de Salazar's ship?    2
How tall is Mount Olympus?                        2
Where is hunting still vital?                     2
What does DRM stand for?                          2
Name: question, dtype: int64

 ----------------------------------------------------------

10 respuestas más populares:

three    231
two      206
four     171
five     133
six       90
2007      87
2006      85
2010      75
seven     71
2009      71
Name: answer, dtype: int64

 ----------------------------------------------------------

10 preguntas más populares (Conjunto de test):

Who conceptualized the piston?                                 

Las preguntas que más se repiten lo hacen a lo más 2 veces cada una. En cuanto a las respuestas, predominan aquellas que son números escritos como palabras, asi como también números escritos como tal los cuales aprecen ser fechas debido a la cantidad de dígitos y la magnitud que presentan.

Lo anterior puede ser en respuesta a un sesgo en la selección de preguntas.

Finalmente, se debe notar que el conjunto de test está compuesto solo por preguntas, lo que significa que, de utilizar los conjuntos de la forma en la que están, tendremos un entrenamiento supervisado pero tendremos que encontrar una métrica nueva para evaluar el desempeño final de la máquina.

## b) Preprocesamiento

Ahora se procederá a preprocesar ambos conjuntos con el objetivo de mejorar el desempeño que tenga la futura máquina al ingerirlos en el entrenamiento.
Se _tokenizarán_ las preguntas y respuestas del conjunto de entrenamiento y de test, no realizando mayor modificación de las palabras dado que después necesitaremos reconstruir las oraciones.

In [0]:
train_questions = [word_tokenize(sentence.lower()) for sentence in train["question"]] #or processing
test_questions = [word_tokenize(sentence.lower()) for sentence in  test["question"]]
train_answers = [word_tokenize(sentence) for sentence in train["answer"]]


## c) Vocabulario

Ahora, se procede a crear un vocabulario para codificar las palabras en las respuestas a generar, esta aproximación nos servirá para paliar el problema mencionado en el punto *a)*, de que no tenemos las respuestas correctas para el conjunto de test.

In [11]:
# Respuestas
vocab_answer = set()
for sentence in train_answers:
  for word in sentence:
    vocab_answer.add(word)
vocab_answer = ["#end"] + list(vocab_answer)
print('Posibles palabras para respuestas: ', len(vocab_answer))
vocabA_indices = {c: i for i, c in enumerate(vocab_answer)}
indices_vocabA = {i: c for i, c in enumerate(vocab_answer)}

# Preguntas: Train
vocab_question = set()
for sentence in train_questions:
  for word in sentence:
    vocab_question.add(word)
vocab_question = ["#end"] + list(vocab_question)
print('Posibles palabras para preguntas (train): ', len(vocab_question))
vocabQTrain_indices = {c: i for i, c in enumerate(vocab_question)}
indices_vocabQTrain = {i: c for i, c in enumerate(vocab_question)}

print('Diferencia en la cantidad de palabras que componen las preguntas y respuestas (train sets): ', 
      abs(len(vocab_answer) - len(vocab_question)))

# Preguntas: Test
vocab_question = set()
for sentence in test_questions:
  for word in sentence:
    vocab_question.add(word)
vocab_question = ["#end"] + list(vocab_question)
print('Posibles palabras para preguntas (test): ', len(vocab_question))
vocabQTest_indices = {c: i for i, c in enumerate(vocab_question)}
indices_vocabQTest = {i: c for i, c in enumerate(vocab_question)}



Posibles palabras para respuestas:  47423
Posibles palabras para preguntas (train):  39482
Diferencia en la cantidad de palabras que componen las preguntas y respuestas (train sets):  7941
Posibles palabras para preguntas (test):  10322


El vocabulario de palabras que componen las respuestas tiene _7941_ elementos más que el que compone las preguntas, esto puede hacer que hayan palabras encontradas en preguntas asociadas a varias palabras de respuesta, haciendo más dificil el discernir la respuesta correcta. Por otro lado, se debe notar la pequeña cantidad de palabras que componen el vocabulario de test.

## d) Codificación de tokens y padding

Aplicaremos una codificación tipo *one-hot vector* sobre los tokens, calcularemos el largo máximo que puede tener una respuesta y una pregunta y reformularemos las secuencias de entrada del modelo agregandoles un padding al final, esto hará que el tamaño de input sea constante. Para las preguntas se rellenará con '0' (recordar que las palabras estan indexadas y tokenizadas), mientras que para las respuestas se rellenará con el carácter definido *'#end'* que indica cuando la pregunta ha sido respondida.



In [0]:
# input and output to onehotvector
X_answers = [[vocabA_indices[palabra] for palabra in sentence] for sentence in train_answers]
X_test_Q = [[vocabQTest_indices[palabra] for palabra in sentence] for sentence in test_questions]
X_train_Q = [[vocabQTrain_indices[palabra] for palabra in sentence] for sentence in train_questions]

# padding
max_input_length = np.max(list(map(len, train_questions)))
max_output_length = np.max(list(map(len, train_answers)))

X_train_Q = sequence.pad_sequences(X_train_Q, maxlen = max_input_length,
                                        padding = 'post', value = 0)
X_test_Q = sequence.pad_sequences(X_test_Q, maxlen = max_input_length,
                                        padding = 'post', value = 0)
X_answers = sequence.pad_sequences(X_answers, maxlen = max_output_length,
                                        padding = 'post', value = vocabA_indices['#end'])

## e) Modelo *Encoder-Decoder* con módulos de atención

Utilizaremos un encoder basado en GRU.

In [0]:
# Encoder-Decoder modelo
length_output = max_output_length
hidden_dim = 128

embedding_vector = 64
encoder_input = Input(shape = (max_input_length, ))
embedded = Embedding(input_dim = len(vocabQTrain_indices), output_dim = embedding_vector,
                    input_length = max_input_length)(encoder_input)
encoder = CuDNNGRU(hidden_dim, return_sequences = True)(embedded)

attention = TimeDistributed(Dense(max_output_length, activation = 'tanh'))(encoder)

# softmax a las atenciones sobre todo T
attention = Permute([2, 1])(attention)
attention = Activation('softmax')(attention)
attention = Permute([2, 1])(attention)


In [0]:
# Aplicacion de la atencion al modelo
def attention_multiply(vects):
  encoder, attention = vects
  return K.batch_dot(attention, encoder, axes = 1)

In [0]:
sent_representation = Lambda(attention_multiply)([encoder, attention])
decoder = CuDNNGRU(hidden_dim, return_sequences= True)(sent_representation)
probabilities = TimeDistributed(Dense(len(vocab_answer), activation = 'softmax'))(decoder)

In [40]:
model = Model(encoder_input, probabilities)
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 60, 64)       2526848     input_2[0][0]                    
__________________________________________________________________________________________________
cu_dnngru_3 (CuDNNGRU)          (None, 60, 128)      74496       embedding_2[0][0]                
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 60, 46)       5934        cu_dnngru_3[0][0]                
__________________________________________________________________________________________________
permute_3 

## f) Entrenamiento del modelo

Se entrenará el modelo con 10 epochs con tamaño de batch 64

In [106]:
X_answers = X_answers.reshape(X_answers.shape[0], X_answers.shape[1],1)
X_answers.shape

(86821, 46, 1)

In [0]:
model.load_weights('attW.h5')

In [16]:
"""
BS = 64
model.fit(X_train_Q, X_answers, epochs = 10, batch_size = BS,
               validation_split = 0.2)"""

Train on 69456 samples, validate on 17365 samples
Epoch 1/10
69456/69456 [==============================] - 356s 5ms/step - loss: 0.9792 - val_loss: 0.8078
Epoch 2/10
69456/69456 [==============================] - 353s 5ms/step - loss: 0.6997 - val_loss: 0.8136
Epoch 3/10
69456/69456 [==============================] - 352s 5ms/step - loss: 0.6826 - val_loss: 0.8227
Epoch 4/10
69456/69456 [==============================] - 352s 5ms/step - loss: 0.6690 - val_loss: 0.8304
Epoch 5/10
69456/69456 [==============================] - 352s 5ms/step - loss: 0.6557 - val_loss: 0.8415
Epoch 6/10
69456/69456 [==============================] - 352s 5ms/step - loss: 0.6383 - val_loss: 0.8402
Epoch 7/10
69456/69456 [==============================] - 352s 5ms/step - loss: 0.6184 - val_loss: 0.8483
Epoch 8/10
69456/69456 [==============================] - 352s 5ms/step - loss: 0.6006 - val_loss: 0.8469
Epoch 9/10
69456/69456 [==============================] - 352s 5ms/step - loss: 0.5834 - val_loss: 0.8

46

## g) Predicción del modelo

Evaluaremos ahora las predicciones del modelo a traves del modelamiento de la distribución de probabilidad de las respuestas, basandonos en la frecuencia de ocurrencia de los tokens encontrados en las mismas.


(1, 46, 47423)

In [20]:
def predict_words(model, example, diversity):
  model = load_model('att.h5')
  example = np.array(example)
  example.reshape((60,))
  prediction = model.predict(example)
  return prediction
  
def determine_answer(answer_probs):
  Y = np.random.binomial(1, answer_probs, 1)
  return Y

  
n = 10
for i in range(n):
  indexs = np.random.randint(0, len(X_test_Q)-2)
  example = X_train_Q[indexs:(indexs+1)]
  indexes_answer = predict_words(model, example, 0.85)
  indexes_answer = determine_answer(indexes_answer)
  question = test['question'][indexs]
  print('Pregunta: ', question)
  answer = ''
  for index in indexes_answer:
    print(index)
    if (indices_vocabA[index] == '#end'): # fin de la oracion
      continue
    else:
      answer += indices_vocabA[index]+' '
  print('Respuesta: ', answer)
print('Los ha predecido todos!')

IndentationError: ignored

In [0]:
inv_map = {v: k for k, v in vocabA_indices.items()}



In [0]:
def determine_answer(answer_probs):
  Y = np.random.binomial(1, answer_probs, 1)
  return Y
from copy import copy
pred = model.predict(X_train_Q[1:2])
pred = pred.reshape(pred.shape[1], pred.shape[2])
#X_train_Q[1:2]



In [196]:
pred.shape

(46, 47423)

In [209]:
resp_indx = []
for i1 in range(pred.shape[0]):
  max_prob = 0.
  max_prob_indx = 0
  for i2 in range(pred.shape[1]):
    prob = pred[i1, i2]
    if(prob >= max_prob):
      max_prob = prob
      max_prob_indx = i2
  resp_indx = resp_indx.append(max_prob_indx)



AttributeError: ignored

## h) Evaluacion del modelo
Para verificar la calidad del modelo, compararemos con el benchmark

138

In [0]:
!python evaluate-v2.0.py dev-v2.0.json predictions

In [0]:
dic_predictions = {}
for example, id_e in zip(Xtest_question, df_test["id"]): # todos los ejemplos
  indexes_answer = predict_words(model, example) # predice palabra en cada instante
  answer = ""
  for index in indexes_answer:
    if(indices_vocabA[index] == '#end'): # Final de la oracion
      continue
    else:
      answer += indices_vocabA[index]+" "
  dic_predictions[id_e] = answer
  contador += 1
  print('Los ha predecido todos!')
  json_save = json.dumps(dic_predictions)
  archivo = open('predictions', 'w')
  archivo.write(json.save)
  archivo.close()